The source code is in the public domain and not licensed or under
copyright. The information and software may be used freely by the public.
As required by 17 U.S.C. 403, third parties producing copyrighted works
consisting predominantly of the material produced by U.S. government
agencies must provide notice with such work(s) identifying the U.S.
Government material incorporated and stating that such material is not
subject to copyright protection.

Derived works shall not identify themselves in a manner that implies an
endorsement by or an affiliation with the Naval Research Laboratory.

RECIPIENT BEARS ALL RISK RELATING TO QUALITY AND PERFORMANCE OF THE
SOFTWARE AND ANY RELATED MATERIALS, AND AGREES TO INDEMNIFY THE NAVAL
RESEARCH LABORATORY FOR ALL THIRD-PARTY CLAIMS RESULTING FROM THE ACTIONS
OF RECIPIENT IN THE USE OF THE SOFTWARE.

In [19]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
from scipy import constants as C
import scipy.interpolate
# The following line provides interactive plots, and requires the ipympl package.
%matplotlib widget

#lbl3 = [r'$x_1$ (mm)',r'$x_2$ (mm)',r'$x_3$ (mm)']
lbl3 = [r'$\varrho$ (mm)',r'$\varphi$',r'$z$ (mm)']
max_microns = 18

mks_length = 0.8e-6/(2*np.pi)
#mks_length = 10.6e-6/(2*np.pi)
base_diagnostic = 'out/test'
uppe_list = glob.glob(base_diagnostic+'*_uppe_wave*.npy')
para_list = glob.glob(base_diagnostic+'*_paraxial_wave*.npy')
total_list = uppe_list + para_list
if 'uppe' in total_list[0]:
    real_field = True
else:
    real_field = False

mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.size'] = 12
l1_mm = 1e3*mks_length
t1_ps = 1e12*mks_length/C.c

def get_freq_time_data(A,ext):
    if real_field:
        # Nodes are like [0,1,2,3] and walls are like [-0.5,0.5,1.5,2.5,3.5]
        # In these examples the user's requested upper bound would be 4 (it is thrown out)
        # Therefore element N/2+1 should be regarded as the central frequency
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        w_nodes = np.linspace(ext[0]+dw/2,ext[1]-dw/2,A.shape[0])
        return dw,wc,tmax,w_nodes
    else:
        # Nodes are like [-2,-1,0,1] and walls are like [-2.5,-1.5,-.5,.5,1.5]
        dw = (ext[1] - ext[0])/A.shape[0]
        wc = 0.5*ext[0] + 0.5*(ext[1] + dw)
        tmax = 2*np.pi/dw
        w_nodes = np.linspace(ext[0]+dw/2,ext[1]-dw/2,A.shape[0])
        return dw,wc,tmax,w_nodes

file = total_list[0]
A = np.load(file)
s = file.split('_')
data_ext_name = s[0] + '_' + s[1] + '_' + s[2] + '_plot_ext.npy'
data_ext = np.load(data_ext_name)

dw,wc,tmax,wn = get_freq_time_data(A,data_ext)
wn += 1e-9

def get_spectral_power(z,rmin):
    # Specializing for cylindrical right now
    # Units will not be correct without applying scale factor externally (see below)
    Ew2 = (np.abs(A[...,z]*wn[...,np.newaxis,np.newaxis])**2)[::-1]
    l_array_nu = (1000*l1_mm*2*np.pi/wn)[::-1]
    l_array = np.linspace(l_array_nu[0],max_microns,Ew2.shape[0])
    for i in range(Ew2.shape[1]):
        for j in range(Ew2.shape[2]):
                fi = scipy.interpolate.interp1d(l_array_nu,Ew2[:,i,j])
                Ew2[:,i,j] = fi(l_array)/l_array
    dr = (data_ext[3]-data_ext[2])/Ew2.shape[1]
    r = np.linspace(data_ext[2]+0.5*dr,data_ext[3]-0.5*dr,Ew2.shape[1])
    dS = (np.pi*(r+0.5*dr)**2 - np.pi*(r-0.5*dr)**2)
    r_idx = np.searchsorted(r,rmin)
    return l_array,np.sum(Ew2[:,r_idx:,0]*dS[np.newaxis,r_idx:],axis=1)

def get_power(z):
    # Specializing for cylindrical right now
    E2 = np.fft.irfft(1j*wn[...,np.newaxis,np.newaxis]*A[...,z],axis=0)**2
    E2 *= (C.m_e*C.c**2/mks_length/C.e)**2
    intensity_mks = E2/377
    dr = (data_ext[3]-data_ext[2])/E2.shape[1]
    r = np.linspace(data_ext[2]+0.5*dr,data_ext[3]-0.5*dr,E2.shape[1])
    dS_mks = mks_length**2*(np.pi*(r+0.5*dr)**2 - np.pi*(r-0.5*dr)**2)
    return np.sum(intensity_mks*dS_mks[np.newaxis,:,np.newaxis],axis=(1,2))

plt.close('all')

In [20]:
power = get_power(0)
U00 = np.sum(power)*tmax*mks_length/C.c/power.shape[0]
print('Pulse energy =',U00)

Pulse energy = 0.00629542394399319


In [8]:
Nz = A.shape[3]
dz_mks = mks_length*(data_ext[7]-data_ext[6])/(Nz-1)
z00 = mks_length*data_ext[6]

# Spectral calibration
ln,power = get_spectral_power(0,0.0)
U00w = np.sum(power)*(ln[1]-ln[0])
spcal = U00/U00w

plt.figure()
z = [6,7,14]
for z0 in z:
    z_mks = z00 + dz_mks*z0
    ln,power = get_spectral_power(z0,0.0)
    plt.plot(ln,spcal*power*1e6,label='{:.1f} cm'.format(z_mks*100))
plt.xlim(1.5,max_microns)
plt.ylim(0,1.0)
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Spectral Power ($\mu$J/um)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: index 6 is out of bounds for axis 3 with size 3

In [ ]:
z = np.linspace(z00,z00+dz_mks*(Nz-1),Nz)
LWIR_energy = np.zeros(Nz)
for z0 in range(15):
    ln,power = get_spectral_power(z0,0.0)
    low_idx = np.searchsorted(ln,6)
    high_idx = np.searchsorted(ln,14)
    LWIR_energy[z0] = np.sum(power[low_idx:high_idx]*spcal*1e6)*(ln[1]-ln[0])

plt.figure()
plt.plot(100*z,LWIR_energy)
plt.xlabel('$z$ (cm)')
plt.ylabel('Energy ($\mu$J)')

In [ ]:
plt.figure()
for rm in list(np.linspace(0.0/mks_length,.002/mks_length,5)):
    ln,power = get_spectral_power(5,rm)
    plt.plot(ln,spcal*power*1e6,label='{:.2f} cm'.format(rm*mks_length*100))
plt.xlim(1.5,14)
plt.ylim(0,1.0)
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Spectral Power ($\mu$J/um)')
plt.legend()